In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

import warnings 
warnings.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data=pd.read_csv('/kaggle/input/logistic-regression/Social_Network_Ads.csv')
data.head()

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
data.drop(['User ID','Purchased'],axis=1,inplace=True)



In [ ]:
data.Gender=[1 if each=='Female' else 0 for each in data.Gender]
data.info()

In [ ]:
y=data.Gender.values
x_data=data.drop(['Gender'],axis=1)

In [ ]:
#normalize
x=(x_data-np.min(x_data))/(np.max(x_data)-np.min(x_data)).values
x

In [ ]:
#train test split 
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)


In [ ]:
#%% train test split
x_train=x_train.T
x_test=x_test.T
y_train=y_train.T
y_test=y_test.T


print('x_train: ',x_train.shape)
print('x_test: ',x_test.shape)
print('y_train: ',y_train.shape)
print('y_test: ',y_test.shape)

In [ ]:
#dimension=2
#initialize parameters and sigmoid function
def initialize_weights_and_bias(dimension):
    w=np.full((dimension,1),0.01)
    b=0.0
    return w,b
def sigmoid(z):
    y_head=1/(1+np.exp(-z))
    return y_head

In [ ]:
#implementing forward and backward propagation

def forward_backward_propagation(w,b,x_train,y_train):
    #forward propagation
    z=np.dot(w.T,x_train)+b
    y_head=sigmoid(z)
    loss=-y_train*np.log(y_head)-(1-y_train)*np.log(1-y_head)
    cost=(np.sum(loss))/x_train.shape[1]
    
    #backward propagation
    derivative_weight=(np.dot(x_train,((y_head-y_train).T)))/x_train.shape[1]
    derivative_bias=np.sum(y_head-y_train)/x_train.shape[1]
    gradients={'derivative_weight':derivative_weight,'derivative_bias':derivative_bias}
    return cost,gradients

In [ ]:
#update(learning) parameters
def update(w,b,x_train,y_train,learning_rate,number_of_iteration):
    cost_list=[]
    cost_list2=[]
    index=[]
    
    #updating(learning) parameters is number_of_iteration times
    for i in range(number_of_iteration):
        cost,gradients=forward_backward_propagation(w,b,x_train,y_train)
        cost_list.append(cost)
        
        #lets update
        w=w-learning_rate*gradients['derivative_weight']
        b=b-learning_rate*gradients['derivative_bias']
        if i%10==0:
            cost_list2.append(cost)
            index.append(i)
            print('Cost after iteration %i: %f'%(i,cost))
    #we update parameters 
    parameters={'weight':w,'bias':b}
    plt.plot(index,cost_list2)
    plt.xticks(index,rotation='vertical')
    plt.xlabel('Number of iteration')
    plt.ylabel('cost')
    plt.show()
    return parameters,gradients,cost_list

In [ ]:
### prediction
def prediction(w,b,x_test):
    z=sigmoid(np.dot(w.T,x_test)+b)
    Y_prediction=np.zeros((1,x_test.shape[1]))
    for i in range(z.shape[1]):
        if z[0,i]<=0.5:
            Y_prediction=0
        else:
            Y_prediction=1
    return Y_prediction
        

In [ ]:
#logistic_regression
def logistic_regression(x_train,y_train,x_test,y_test,learning_rate,num_iterations):
    
    dimension=x_train.shape[0]
    w,b=initialize_weights_and_bias(dimension)
    
    parameters,gradients,cost_list=update(w,b,x_train,y_train,learning_rate,num_iterations)
    
    y_prediction_test=prediction(parameters['weight'],parameters['bias'],x_test)
    #print test errors
    print("test accuracy: {} %".format(100 - np.mean(np.abs(y_prediction_test - y_test)) * 100))
    

In [ ]:
#lets see
logistic_regression(x_train,y_train,x_test,y_test,learning_rate=2,num_iterations=320)

In [ ]:
#sklearn with LR
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()
lr.fit(x_train.T,y_train.T)
print('test accuracy: {}'.format(lr.score(x_test.T,y_test.T)))